In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 以下为正文！

## 读数据集

In [ ]:
import os
from tqdm import tqdm
from dataset import MyDataset
data_path = "dataset\seg_train"
test_path = "dataset\seg_test"
dataset = MyDataset(data_path)
testset = MyDataset(test_path)

## SIFT

In [ ]:
import cv2

sift = cv2.SIFT_create()
def sift_algorithm(image: cv2.Mat) -> np.ndarray:
    
    # 改为灰度图
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 创建SIFT对象实例
    
    # sift.detect()作用：在图像中找到关键点。
    # 如果只想搜索图像的一部分，可以传递mask。
    # 每个关键点都是一个特殊的结构，它有许多属性，比如（x，y）坐标、有意义邻域的大小、指定其方向的角度、指定关键点强度的响应等。
    # kp = sift.detect(gray, None)

    # 现在已经找到关键点，就可以使用sift.compute()根据关键点计算描述符。
    # 最终得到的kp是一个关键点元组，des是一个形状（关键点数量）×128的numpy数组。
    # kp, des = sift.compute(gray, kp)

    # 刚刚我们分两步，先找到关键点、再计算描述符。
    # 可以直接一步实现，即使用sift.detectAndCompute()方法。
    kp, des = sift.detectAndCompute(image_gray, None)
    return des

from sklearn.cluster import KMeans
import random
def get_sift_clustering() -> KMeans:
    '''
    根据上述计算得出的所有特征点和特征向量，训练一个聚类分类器。
    '''
    keypoint_vectors = []
    for image, label in tqdm(dataset):
        vectors = sift_algorithm(image)
        if vectors is None: # 可能为None
            continue
        for v in vectors:
            keypoint_vectors.append(v)
    random.shuffle(keypoint_vectors)
    print("Total key points:", len(keypoint_vectors))
    sift_clustering = KMeans(n_clusters=100)
    sift_clustering.fit(keypoint_vectors[:10000])
    return sift_clustering

def get_wordbag_vector(image: cv2.Mat, clustering: KMeans) -> np.ndarray:
    '''
    用聚类算法给所有特征点分类，然后用词袋模型得到单张图片的词袋向量。
    '''
    vectors = sift_algorithm(image)
    feature_vector = np.zeros(100)
    if vectors is None: # 可能为None
        return feature_vector
    labels = clustering.predict(np.asarray(vectors, dtype=float))
    for label in labels:
        feature_vector[label] += 1
    return feature_vector
        

In [ ]:
# Test
from utils import label_int2str
image, label = dataset[0]
features = sift_algorithm(image).flatten()
print("Label: %s (%s), features:"%(label, label_int2str(label)))
print(features)
print("Shape:", features.shape)

## Hog

In [ ]:
winSize = (8,8)
blockSize = (8,8)
blockStride = (4,4)
cellSize = (8,8)
nbins = 5

hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)

In [ ]:
# Test
image, label = dataset[0]
print(image.shape)
features = hog.compute(image)
print("Label: %s (%s), features:"%(label, label_int2str(label)))
print(features)
print("Shape:", features.shape)

## Preprocess data

In [ ]:
X_sift_train = []
X_hog_train = []
y_train = []

sift_clustering = get_sift_clustering()
import pickle
pickle.dump(sift_clustering, open("./outputs/sift_clustering", 'wb'))

for image, label in tqdm(dataset):
    X_sift_train.append(get_wordbag_vector(image, sift_clustering))
    X_hog_train.append(hog.compute(image))
    y_train.append(label)
X_sift_train = np.stack(X_sift_train)
X_hog_train = np.stack(X_hog_train)
y_train = np.array(y_train)
os.makedirs("./dataset/processed", exist_ok=True)
np.save("./dataset/processed/X_sift_train.npy", X_sift_train)
np.save("./dataset/processed/X_hog_train.npy", X_hog_train)
np.save("./dataset/processed/y_train.npy", y_train)

X_sift_test = []
X_hog_test = []
y_test = []
for image, label in tqdm(dataset):
    X_sift_test.append(get_wordbag_vector(image, sift_clustering))
    X_hog_test.append(hog.compute(image))
    y_test.append(label)
X_sift_test = np.stack(X_sift_test)
X_hog_test = np.stack(X_hog_test)
y_test = np.array(y_test)
os.makedirs("./dataset/processed", exist_ok=True)
np.save("./dataset/processed/X_sift_test.npy", X_sift_test)
np.save("./dataset/processed/X_hog_test.npy", X_hog_test)
np.save("./dataset/processed/y_test.npy", y_test)

## Classifiers

In [ ]:
from sklearn.svm import LinearSVC, SVC

USED_EXTRACTOR = "sift"
if USED_EXTRACTOR=="hog":
    X_train_used = X_hog_train
    X_test_used = X_hog_test
elif USED_EXTRACTOR=="sift":
    X_train_used = X_sift_train
    X_test_used = X_sift_test
else:
    raise ValueError("Either hog or sift! ")
print(X_train_used.shape)

# X_train_used = np.load("./dataset/processed/X_sift_train.npy")
# X_test_used = np.load("./dataset/processed/X_sift_test.npy")
# For emergency. 

# SVM
svm = LinearSVC()
svm.fit(X_train_used, y_train)
print("SVM trained. ")
svm_accuracy = svm.score(X_test_used, y_test)
print("Accuracy:", svm_accuracy)

# Kernel-SVM
kernel_svm = SVC()
kernel_svm.fit(X_train_used, y_train)
print("Kernel-SVM trained. ")
kernel_svm_accuracy = kernel_svm.score(X_test_used, y_test)
print("Accuracy:", kernel_svm_accuracy)


In [ ]:
from sklearn.cluster import KMeans

# k-means
k_means = KMeans(n_clusters=6)
k_means.fit(X_train_used)
print("K-Means trained. ")
from utils import clustering_accuracy
k_means_predictions = k_means.predict(X_test_used)
clustering_accuracy(y_test, k_means_predictions)

In [ ]:
# Save models
import pickle
pickle.dump(svm, open("./outputs/svm_%s"%USED_EXTRACTOR, 'wb'))
pickle.dump(kernel_svm, open("./outputs/kernel_svm_%s"%USED_EXTRACTOR, 'wb'))
pickle.dump(k_means, open("./outputs/k_means_%s"%USED_EXTRACTOR, 'wb'))

## 以上是预处理数据及训练模型过程。以下仅检验模型正确率。

In [1]:
# Load models
import pickle
svm_sift = pickle.load(open("./outputs/svm_sift", 'rb'))
kernel_svm_sift = pickle.load(open("./outputs/kernel_svm_sift", 'rb'))
k_means_sift = pickle.load(open("./outputs/k_means_sift", 'rb'))
svm_hog = pickle.load(open("./outputs/svm_hog", 'rb'))
kernel_svm_hog = pickle.load(open("./outputs/kernel_svm_hog", 'rb'))
k_means_hog = pickle.load(open("./outputs/k_means_hog", 'rb'))

# sift_clustering = pickle.load(open("./outputs/sift_clustering", 'rb'))

# Load processed data
import numpy as np
X_sift_test = np.load("./dataset/processed/X_sift_test.npy")
X_hog_test = np.load("./dataset/processed/X_hog_test.npy")
y_test = np.load("./dataset/processed/y_test.npy")


# Test accuracies
from utils import clustering_accuracy
print("Accuracy of SIFT & SVM:", svm_sift.score(X_sift_test, y_test))
print("Accuracy of SIFT & Kernel-SVM:", kernel_svm_sift.score(X_sift_test, y_test))
print("Accuracy of SIFT & k-means:", clustering_accuracy(y_test, k_means_sift.predict(X_sift_test)))
print("Accuracy of HOG & SVM:", svm_hog.score(X_hog_test, y_test))
print("Accuracy of HOG & Kernel-SVM:", kernel_svm_hog.score(X_hog_test, y_test))
print("Accuracy of HOG & k-means:", clustering_accuracy(y_test, k_means_hog.predict(X_hog_test)))


Accuracy of SIFT & SVM: 0.5571285571285571
Accuracy of SIFT & Kernel-SVM: 0.7277992277992278
Accuracy of SIFT & k-means: 0.33697983697983697
Accuracy of HOG & SVM: 0.7166452166452166
Accuracy of HOG & Kernel-SVM: 0.8787358787358788
Accuracy of HOG & k-means: 0.389031889031889
